# Supporting Documents and Variables

In [2]:
# Loading Libraries needed
import os
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import re
import matplotlib

/usr/local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## State FIPS codes

These were taken from the [US Census Bureau](https://www.census.gov/library/reference/code-lists/ansi.html#state). This were to request data at a state by state level for their ACS variables. [Specifically taken from this URL](https://www2.census.gov/geo/docs/reference/state.txt)

In [15]:
# Making a request for the State FIPS codes
# Note these are pipe ie "|" separated
state_fips_url = "https://www2.census.gov/geo/docs/reference/state.txt"
resp = requests.get(state_fips_url)
print(resp.status_code)
state_fips = resp.text

state_fips_outpath = "data/support/state_fips_raw.txt"
state_fips_f = open(state_fips_outpath, "w")
state_fips_f.write(state_fips)
state_fips_f.close()

# Formatting data so that FIPS column is padded with 0 when is a single digit
state_fips_df = pd.read_csv(state_fips_outpath, delimiter="|", dtype={"STATE": str})
state_fips_df["STATE"] = np.where(
    state_fips_df["STATE"].str.len() == 1,
    '0' + state_fips_df["STATE"],
    state_fips_df["STATE"]
)

state_fips_df.to_csv("data/support/state_fips.csv", index=False)

200


## Geographic References

In [5]:
# Use census tract metadata file as base dataframe for joins
# Taken from: https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html
census_tracts = gpd.read_file("data/geography/cb_2021_us_tract_500k.zip", dtype = {
   "STATEFP": str,
   "COUNTYFP": str,
   "TRACTCE": str,
   "GEOID": str
})

unwanted_states = [
	"60", # American Samoa
  "66", # Guam
  "69", # Northern Mariana Islands
  "72", # Puerto Rico
  "78", # Virgin Islands
  "15", # Hawaii
  "02",  # Alaska
]

census_tracts = census_tracts[~census_tracts.STATEFP.isin(unwanted_states)]
census_tracts.to_file("clean_data/cb_2021_us_tract_500k_filtered.json", driver='GeoJSON')

In [ ]:
# Gets all shapefiles for all states and territories

# Gets list of all state fips codes
state_fips_df = pd.read_csv("data/support/state_fips.csv", dtype = {"STATE": str})
state_fips = list(state_fips_df["STATE"])
base_shp_url = "https://www2.census.gov/geo/tiger/TIGER2021/TRACT/"

geo_fp = "data/geography"

# Iterate through all state fips codes and get their shapefile
for curr_state in state_fips:

   # Build current shapefile URL
   curr_shp_name = f"tl_2021_{curr_state}_tract.zip"
   curr_shp_url = base_shp_url + curr_shp_name
   shp_resp = requests.get(curr_shp_url)
   
   # Write data out
   if shp_resp.status_code == 200:
      outfile = os.path.join(geo_fp, curr_shp_name)
      f = open(outfile, "wb")
      f.write(shp_resp.content)
      f.close()
      print(f"Done {curr_shp_name}")
   # Print error message otherwise
   else:
      print(f"Error in retreiving {curr_state} data")

In [ ]:
# Gets list of all census tracts for all census tracts from shapefiles
geo_fp = "data/geography/2021"
shp_files = os.listdir(geo_fp)

census_tract_df = None

# iterate through all shapefiles and compile census tract metadata
for shp_file in shp_files:
   # build shapefile path and read file
   curr_fp = os.path.join(geo_fp, shp_file)
   curr_gpd = gpd.read_file(curr_fp)

   # limit to relevant columns
   curr_gpd = curr_gpd[["STATEFP", "COUNTYFP", "TRACTCE", "GEOID", "geometry"]]

   # add to overall census tract dataframe
   if not (census_tract_df is None):
      census_tract_df = pd.concat([census_tract_df, curr_gpd])
   else:
      census_tract_df = curr_gpd

   print(f"{shp_file} done!")

# Write out overall census tract dataframe
#census_tract_df.to_csv(os.path.join(geo_fp, "tract_metadata.csv"), index = False)
census_tract_df = gpd.GeoDataFrame(
    census_tract_df,
    geometry="geometry"
)
census_tract_df.to_file(os.path.join(geo_fp, "updated_complete_tract.geojson"), driver='GeoJSON')
print("Done writing census tract metadata!")

## US Census API Interaction
In order to interact with the US Census Bureau API you need to sign up for an [API key here](https://api.census.gov/data/key_signup.html).

## US Census API Interaction
In order to interact with the US Census Bureau API you need to sign up for an [API key here](https://api.census.gov/data/key_signup.html).